https://quantiacs.com/documentation/en/user_guide/local_development.html

# The environment

The next three code cells contains commands that will configure your Coogle Colab environment. 

When you transfer the strategy to Quantiacs, remove these cells.

They are not relevant for the Quantiacs platform.

At first, setup the toolbox from github using pip:

In [ ]:
###DEBUG###

! pip install git+https://github.com/quantiacs/toolbox.git

# decrease height
from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/quantiacs/toolbox.git to /tmp/pip-req-build-yveykhtp
  Running command git clone -q https://github.com/quantiacs/toolbox.git /tmp/pip-req-build-yveykhtp
     |████████████████████████████████| 9.9 MB 4.7 MB/s 
     |████████████████████████████████| 355 kB 41.0 MB/s 
     |████████████████████████████████| 3.4 MB 32.1 MB/s 
     |████████████████████████████████| 25.3 MB 1.5 MB/s 
  Created wheel for qnt: filename=qnt-0.0.272-py3-none-any.whl size=114264 sha256=d1b3a0ac3ed4968a3f6a38913d440158b8d00373252e5d943bd34e0f72273a4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-85o8tari/wheels/eb/4b/a6/2941ec3b0a208a66ffa127c74cbcddbff9453f5ed798ee9be2
Successfully built qnt
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: llvmlite
    Fou

<IPython.core.display.Javascript object>

Then install TA-Lib (indicators library) if you need it.

Instead of TA-Lib you can use qnt.ta or another library. In this case, skip the next cell.

Note that the installation can take several minutes.

In [ ]:
###DEBUG###
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
! make
!make install
!pip install Ta-Lib

# test import
import talib

# decrease height
from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))

--2022-09-30 09:27:35--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-30 09:27:35--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://versaweb.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-30 09:27:35--  http://versaweb.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving versaweb.dl.s

<IPython.core.display.Javascript object>

Finally specify the api key and disable interactive charts.

You can find the api key in [your profile](https://quantiacs.com/personalpage/homepage).

We disable interactive charts in the library because interact+plotly do not work correctly in Google Colab.

In [ ]:
###DEBUG###
import os

os.environ['API_KEY'] = "" 
os.environ['NONINTERACT'] = 'True' 

# The strategy

The next cell contains the strategy code itself.

In [ ]:
import xarray as xr

import qnt.stats as qnstats
import qnt.data as qndata
import qnt.output as qnout
import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.graph as qngraph


def strategy(data):
    # calc weights:
    close = data.sel(field="close")
    ma_slow = qnta.lwma(close, 250)
    ma_fast = qnta.lwma(close, 50)
    return xr.where(ma_fast > ma_slow, 1, -1)


# # SINGLE-PASS
# # ---
# # This is fast implementation, but it can easily become looking forward (common problem).
# # Use this approach for research and optimization. And use multi-pass to detect looking forward.
# data = qndata.futures.load_data(min_date="2005-01-01")  # load data

# output = strategy(data)
# output = qnout.clean(output, data) # fix common errors

# qnout.check(output, data) # check that weights are correct:
# qnout.write(output) # write results, necessary for submission:

# stats = qnstats.calc_stat(data, output.sel(time=slice("2006-01-01", None))) # calc stats
# print(stats.to_pandas().tail())
# # qngraph.make_major_plots(stats) # works in jupyter
# # ---


# MULTI-PASS
# ---
# Use this approach to make sure that your strategy is not looking forward.
weights = qnbt.backtest(
    competition_type="futures",  # Futures contest
    lookback_period=365,  # lookback in calendar days
    start_date="2006-01-01",
    strategy=strategy,
    analyze=True,
    build_plots=True
)

# ---

NOTICE: The environment variable DATA_BASE_URL was not specified. The default value is 'https://data-api.quantiacs.io/'
NOTICE: The environment variable CACHE_RETENTION was not specified. The default value is '7'
NOTICE: The environment variable CACHE_DIR was not specified. The default value is 'data-cache'


Run last pass...
Load data...


| |                    #                        | 9096873 Elapsed Time: 0:00:02


Run strategy...
Load data for cleanup...
Output cleaning...
fix uniq
ffill if the current price is None...
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.
Write result...


NOTICE: The environment variable OUTPUT_PATH was not specified. The default value is 'fractions.nc.gz'


...

The number of systems with a larger Sharpe ratio and correlation larger than 0.9: 1
The max correlation value (with systems with a larger Sharpe ratio): 0.9424588470673012
Current sharpe ratio(3y): 0.1973199900954931

Correlated examples:

Name                                  Coefficient    Sharpe ratio
----------------------------------  -------------  --------------
Futures Trend-Following - disabled       0.942459         0.32768
---
Align...
Calc global stats...
---
Calc stats per asset...
Build plots...
---
Output:


asset,F_AD,F_AE,F_AG,F_AH,F_AX,F_BC,F_BG,F_BO,F_BP,F_C
time,,,,,,,,,,
2022-09-16,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,0.014085,-0.014085,-0.014085,0.014085
2022-09-19,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,0.014085,-0.014085,-0.014085,0.014085
2022-09-20,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,0.014085,-0.014085,-0.014085,0.014085
2022-09-21,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,0.014085,-0.014085,-0.014085,0.014085
2022-09-22,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,0.014085,-0.014085,-0.014085,0.014085
2022-09-23,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,0.014085,-0.014085,-0.014085,0.014085
2022-09-26,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,0.014085
2022-09-27,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,0.014085
2022-09-28,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,-0.014085,0.014085


Stats:


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2022-09-16,1.072425,0.001792,0.070292,-0.189259,-0.321962,0.059631,0.004192,-0.577465,71.0,0.027023,102.799122
2022-09-19,1.072743,0.000297,0.070284,-0.189018,-0.321962,0.059878,0.004208,-0.577465,71.0,0.027024,102.769273
2022-09-20,1.077971,0.004873,0.070286,-0.185066,-0.321962,0.064016,0.004499,-0.577465,71.0,0.027020,102.769273
2022-09-21,1.077824,-0.000136,0.070278,-0.185177,-0.321962,0.063893,0.004490,-0.577465,71.0,0.027017,102.769273
2022-09-22,1.081683,0.003580,0.070275,-0.182260,-0.321962,0.066934,0.004704,-0.577465,71.0,0.027013,102.769273
2022-09-23,1.094121,0.011499,0.070323,-0.172857,-0.321962,0.076637,0.005389,-0.605634,71.0,0.027011,102.769273
2022-09-26,1.097487,0.003076,0.070319,-0.170312,-0.321962,0.079247,0.005573,-0.661972,71.0,0.027016,102.741184
2022-09-27,1.098144,0.000599,0.070311,-0.169815,-0.321962,0.079750,0.005607,-0.661972,71.0,0.027024,103.004444
2022-09-28,1.092265,-0.005354,0.070315,-0.174260,-0.321962,0.075143,0.005284,-0.661972,71.0,0.027020,103.004444


---


#Submit your strategy

Finally, you can transfer this notebook to Quantiacs.

Just create a new strategy notebook in [your area](https://quantiacs.com/personalpage/strategies) and upload this notebook to this new environment (replace `strategy.ipynb`).

Don't forget to **remove code cells from "The environment"** section.

If you have any special external dependency which is absent in Quantiacs jupyter environment, add their installation code to `init.ipynb`.

Then test and submit your strategy.